In [4]:
# original dataset
original_data = '/home/sahau24/csc790project/CSC_790_Synthetic_Data/penguins_original.csv'


## unprocessed_df = original unprocess data

In [5]:
import pandas as pd

In [6]:
unprocessed_df = pd.read_csv(original_data)

In [7]:
# proportion of data in sex column
print(unprocessed_df['sex'].value_counts(normalize=False))

sex
MALE      168
FEMALE    165
.           1
Name: count, dtype: int64


In [9]:
#missing values in sex column
print(len(unprocessed_df[unprocessed_df['sex'].isnull()]))

10


In [10]:
unprocessed_df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [11]:
unprocessed_df["sex"].value_counts(normalize=False)

sex
MALE      168
FEMALE    165
.           1
Name: count, dtype: int64

In [12]:
unprocessed_df["species"].value_counts()

species
Adelie       152
Gentoo       124
Chinstrap     68
Name: count, dtype: int64

In [13]:
# missing values in sex column
len(unprocessed_df[unprocessed_df['sex'].isnull()])

10

In [14]:
# unprocess df with missing values in any columns
unprocessed_df[unprocessed_df.isnull().any(axis=1)] 
print(len(unprocessed_df[unprocessed_df.isnull().any(axis=1)]))


10


In [15]:
import pandas as pd
fresh_df = unprocessed_df.dropna()
fresh_df.shape

(334, 7)

In [16]:
# preprocess the catgorical features
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
categorical_features = ['species', 'island', 'sex']
for col in categorical_features:
    le = LabelEncoder()
    fresh_df[col] = le.fit_transform(fresh_df[col])

/tmp/ipykernel_1087697/3052189557.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fresh_df[col] = le.fit_transform(fresh_df[col])
/tmp/ipykernel_1087697/3052189557.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fresh_df[col] = le.fit_transform(fresh_df[col])
/tmp/ipykernel_1087697/3052189557.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [17]:
fresh_df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.1,18.7,181.0,3750.0,2
1,0,2,39.5,17.4,186.0,3800.0,1
2,0,2,40.3,18.0,195.0,3250.0,1
4,0,2,36.7,19.3,193.0,3450.0,1
5,0,2,39.3,20.6,190.0,3650.0,2


In [18]:
#unique values in sex column
fresh_df["sex"].unique()

array([2, 1, 0])

In [19]:
# remove 0 unique values from sex column
fresh_df = fresh_df[fresh_df["sex"] != 0]
fresh_df.shape

(333, 7)

 1 - male 
 2 - female 
 3- dot
 0 - Nan

In [20]:
# Split the dataset into train, val and test sets
from sklearn.model_selection import train_test_split

train_val_df, test_df = train_test_split(fresh_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2
print(f"Train shape: {train_df.shape}, Val shape: {val_df.shape}, Test shape: {test_df.shape}")
# Separate features and target variable
X_train = train_df.drop(columns=['species'])
y_train = train_df['species']
X_val = val_df.drop(columns=['species'])
y_val = val_df['species']
X_test = test_df.drop(columns=['species'])
y_test = test_df['species'] 


Train shape: (199, 7), Val shape: (67, 7), Test shape: (67, 7)


In [21]:
# Run Random Forest Classifier on original data
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score   

rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)
# validation set
print("Validation Set Results:")
y_val_pred = rf.predict(X_val)
print("Random Forest Classifier on Original Data")
print(classification_report(y_val, y_val_pred))
print("Accuracy:", accuracy_score(y_val, y_val_pred))
# area under ROC curve      
from sklearn.metrics import roc_auc_score
y_val_prob = rf.predict_proba(X_val)
roc_auc = roc_auc_score(y_val, y_val_prob, multi_class='ovr')
print("ROC AUC:", roc_auc)

# test set  
print("________________________________")
print("Test Set Results:")
y_pred = rf.predict(X_test)
print("Random Forest Classifier on Original Data")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
# area under ROC curve
from sklearn.metrics import roc_auc_score
y_prob = rf.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')
print("ROC AUC:", roc_auc)




Validation Set Results:
Random Forest Classifier on Original Data
              precision    recall  f1-score   support

           0       0.97      0.94      0.95        32
           1       0.87      0.93      0.90        14
           2       1.00      1.00      1.00        21

    accuracy                           0.96        67
   macro avg       0.94      0.96      0.95        67
weighted avg       0.96      0.96      0.96        67

Accuracy: 0.9552238805970149
ROC AUC: 0.9914869721473495
________________________________
Test Set Results:
Random Forest Classifier on Original Data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        23

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67

Accuracy: 1.0
ROC AUC:

In [22]:
# run histogram-based gradient boosting classifier on original data
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
hgb.fit(X_train, y_train)
# validation set
print("Validation Set Results:")        
y_val_pred_hgb = hgb.predict(X_val)
print("Histogram-based Gradient Boosting Classifier on Original Data")
print(classification_report(y_val, y_val_pred_hgb))
print("Accuracy:", accuracy_score(y_val, y_val_pred_hgb))
# area under ROC curve

y_val_prob_hgb = hgb.predict_proba(X_val)
roc_auc_hgb = roc_auc_score(y_val, y_val_prob_hgb, multi_class='ovr')
print("ROC AUC:", roc_auc_hgb)
# test set
print("---------------------")
print("Test Set Results:")  
y_pred_hgb = hgb.predict(X_test)
print("Histogram-based Gradient Boosting Classifier on Original Data")
print(classification_report(y_test, y_pred_hgb))
print("Accuracy:", accuracy_score(y_test, y_pred_hgb))
# area under ROC curve
y_prob_hgb = hgb.predict_proba(X_test)
roc_auc_hgb = roc_auc_score(y_test, y_prob_hgb, multi_class='ovr')
print("ROC AUC:", roc_auc_hgb)  

Validation Set Results:
Histogram-based Gradient Boosting Classifier on Original Data
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        32
           1       0.93      0.93      0.93        14
           2       1.00      1.00      1.00        21

    accuracy                           0.97        67
   macro avg       0.97      0.97      0.97        67
weighted avg       0.97      0.97      0.97        67

Accuracy: 0.9701492537313433
ROC AUC: 0.9977594339622642
---------------------
Test Set Results:
Histogram-based Gradient Boosting Classifier on Original Data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        23

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00     

In [23]:
# Logistic Regression on original data
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=20000)          

log_reg.fit(X_train, y_train)
# validation set
print("Validation Set Results:")
y_val_pred = log_reg.predict(X_val)
print("Logistic Regression on Original Data")
print(classification_report(y_val, y_val_pred))
print("Accuracy:", accuracy_score(y_val, y_val_pred))
# area under ROC curve      
y_val_prob = log_reg.predict_proba(X_val)       
roc_auc = roc_auc_score(y_val, y_val_prob, multi_class='ovr')
print("ROC AUC:", roc_auc)
# test set  
print("________________________________")
print("Test Set Results:")
y_pred = log_reg.predict(X_test)
print("Logistic Regression on Original Data")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
# area under ROC curve
y_prob = log_reg.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr')
print("ROC AUC:", roc_auc)  

Validation Set Results:
Logistic Regression on Original Data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        32
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00        21

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67

Accuracy: 1.0
ROC AUC: 1.0
________________________________
Test Set Results:
Logistic Regression on Original Data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        23

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67

Accuracy: 1.0
ROC AUC: 1.0
